# 摘要
人工處理，每一篇約要耗費3分鐘的時間處理到好。

In [1]:
import os, pandas, numpy, re
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.shared import RGBColor
from docx.enum.text import WD_LINE_SPACING
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import Inches


## 函數庫

In [ ]:
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

In [ ]:
# 標題設置。
def title_set(doc, text, level, size, line_spacing, text_font) : 
    title = doc.add_heading(level = level)
    title_text = title.add_run(text)
    title_text._element.rPr.rFonts.set(qn('w:eastAsia'), text_font)
    title.style.font.color.rgb = RGBColor(0, 0, 0) # 設為黑色。
    title.style.font.underline = False # 去除標題之底線。
    title.style.font.size = Pt(size)
    title.paragraph_format.line_spacing = line_spacing 


## Step_1

In [2]:
airitlibrary_df = pandas.read_csv(os_path(["database", "華藝資料庫.csv"]))
airitlibrary_df.head(-3)

,author,paper_name,periodicals_name,periodicals_detail,periodicals_date,periodicals_page,abstract,citations
0,曹志明,設計社會學,設計研究,7期,2007,P42-48,美國心理學家馬斯洛（Abraham Maslow）說人的需要有七層，即：一、食物、水；二、安...,2.0
1,義守大學《人文與社會學報》編輯小組,義守大學《人文與社會學報》第三卷第一期編後語,人文與社會學報,3卷1期,2012,P143-143,頁面尚無資料,12.0
2,彼得•貝爾(PeterBaehr),"Philosophy,Sociology,andtheIntelligentsia:Hann...",人文及社會科學集刊,19卷3期,2007,P341-373,本文將檢視漢娜•鄂蘭(Hannah Arendt)對《意識型態與烏托邦》(Ideology ...,NaN
3,關永中(Wing-ChungKwan),謝勒知識社會學的背景－文化社會學,臺大文史哲學報,60期,2004,P265-295,謝勒論「知識社會學」，首先把它放在「文化社會學」的背景內作探討。而在「文化社會學」上、尤凸顯...,NaN
4,陳宗文(Tzung-WenChen),社會學之眼：在馬內的《鐵路》中重新發現社會學,歐美研究,49卷1期,2019,P75-141,社會學之眼存在社會學知識產生之前，或社會學成為一門學科之前，用以察驗與開啟社會學問題意識，挑...,NaN
5,蔡毓智(Yuh-JyhTsai),社會網絡：社會學研究的新取向,思與言：人文與社會科學期刊,46卷1期,2008,P1-33,社會網絡(social networks)理論及其分析方法，已經逐漸為社會科學界加以應用，並...,NaN
6,李宏仁,調解制度與社會學之關聯,人文社會學報,3期,2004,P139-150,糾紛既爲人類社會所無可避免的現象，而調解又係在未起訴或未判決前從中調停排解，使紛爭獲得合理的...,NaN


## Step_2

In [6]:
# 創建 word 檔。
from docx import Document
document = Document()

# 設定 docx 新細明體為內文之預設字體。
document.styles['Normal'].font.name = "Times New Roman"
document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u"新細明體")

# 設置標題，level 為標題層級。
title_chinese_reference_3 = document.add_heading(level = 3)
title_chinese_reference_run = title_chinese_reference_3.add_run("中文參考資料")

# 此處需再設定一次，原因有要，其一、標題不算是內文，其二、標題無法直接更改預設，僅能在這裡改。
title_chinese_reference_run.font.name = "Times New Roman"
title_chinese_reference_run._element.rPr.rFonts.set(qn('w:eastAsia'), u'新細明體') 

# 標題段落設定
title_chinese_reference_3.style.font.color.rgb = RGBColor(0, 0, 0) # 設為黑色。
title_chinese_reference_3.style.font.underline = False # 去除標題之底線。
title_chinese_reference_3.style.font.size = Pt(12)
title_chinese_reference_3.paragraph_format.line_spacing = 2 # 標題行距為2。

for index, row in airitlibrary_df.iterrows() : 

    author = re.sub("\([^\)]*\)", "", row["author"]) # 先匹配的理由在於，後匹配會與 X期X卷，的格式衝突，進而導致多刪除資訊。

    text = author + "，"+ str(row["periodicals_date"]) + "，〈"+ row["paper_name"] + "〉。《" + row["periodicals_name"] + "》"+ row["periodicals_detail"] + " : " + row["periodicals_page"] + "。"

    # 將 X 卷 X期，的資料格式整理成台灣社會學刊的格式。
    if "卷" in text : # 預期格式僅有，X卷X期、X卷、X期，三種。
        if "期" in text : 
            text = re.sub("(\d+)卷(\d+)期", r"\1(\2)", text)
        else : 
            text = re.sub("(\d+)卷", r"\1", text)
    else : 
        text = re.sub("(\d+)期", r"\1", text)
    
    # 將 P頁的 P字元去除。
    text = re.sub("P", "", text)
    
    # 寫入文字
    # 建構段落
    paragraph = document.add_paragraph()
    paragraph_format = paragraph.paragraph_format
    paragraph_format.left_indent = Inches(0.5)  # 整個段落向左缩进0.5英寸。
    paragraph_format.first_line_indent = Inches(-0.5)  # 第一行向右縮進 0.5英吋。
    paragraph_format.line_spacing_rule = WD_LINE_SPACING.MULTIPLE
    paragraph_format.line_spacing = 1.5 # 每一行文字的行距。
    paragraph_format.space_after = Pt(0) # 與後段的距離，修改為零。
    paragraph.add_run(text)
    
    # 修改每一句，寫入文字之字體、大小、。
    for run in paragraph.runs:
        run.font.size = Pt(12)

# 輸出檔案
os.makedirs(os_path(["Word_檔", "text.docx"]), mode=0o777, exist_ok = True)
document.save(os_path(["Word_檔", "text.docx"]))



## 測試區